In [1]:
import pandas as pd
import json
import pymongo
from pymongo import MongoClient

In [2]:
# Read BLS CSV umemployment file into dataframe using pandas

county_unemployment = pd.read_csv("2006_2016_unemployment_by_county.csv")
county_unemployment.head()

,laus_code,state_fips_code,county_fips_code,county_state,year,labor_force,employed,unemployed,unemployment_rate
0,CN0100100000000,1,1,"Autauga , AL",2006,"24,398","23,585",813,3.3
1,CN0100300000000,1,3,"Baldwin , AL",2006,"79,711","77,147","2,564",3.2
2,CN0100500000000,1,5,"Barbour , AL",2006,"10,705","10,096",609,5.7
3,CN0100700000000,1,7,"Bibb , AL",2006,"8,850","8,477",373,4.2
4,CN0100900000000,1,9,"Blount , AL",2006,"26,770","25,902",868,3.2


In [3]:
# Split BLS combined county_state column in dataframe into individual county, state columns

county_unemployment_df = pd.DataFrame(county_unemployment)
county_unemployment_df[['county','state']]= county_unemployment_df.county_state.str.split(",",expand=True,)

In [4]:
# Drop original BLS county_state column

clean_county_df = county_unemployment_df.drop('county_state',axis=1)
clean_county_df = clean_county_df.dropna()

In [5]:
# Drop Puerto Rico from dataframe

clean_county_df = clean_county_df[~clean_county_df['state'].str.contains('PR')]
clean_county_df.head()

,laus_code,state_fips_code,county_fips_code,year,labor_force,employed,unemployed,unemployment_rate,county,state
0,CN0100100000000,1,1,2006,"24,398","23,585",813,3.3,Autauga,AL
1,CN0100300000000,1,3,2006,"79,711","77,147","2,564",3.2,Baldwin,AL
2,CN0100500000000,1,5,2006,"10,705","10,096",609,5.7,Barbour,AL
3,CN0100700000000,1,7,2006,"8,850","8,477",373,4.2,Bibb,AL
4,CN0100900000000,1,9,2006,"26,770","25,902",868,3.2,Blount,AL


In [6]:
# Sort cleaned_county_df

sorted_county_df = clean_county_df.sort_values(['unemployment_rate'], ascending=False)
sorted_county_df.head()

,laus_code,state_fips_code,county_fips_code,year,labor_force,employed,unemployed,unemployment_rate,county,state
16278,CN0602500000000,6,25,2011,"79,456","56,524","22,932",28.9,Imperial County,CA
13059,CN0602500000000,6,25,2010,"78,665","55,978","22,687",28.8,Imperial County,CA
19497,CN0602500000000,6,25,2012,"79,265","57,586","21,679",27.4,Imperial County,CA
9840,CN0602500000000,6,25,2009,"75,569","54,889","20,680",27.4,Imperial County,CA
12926,CN0113100000000,1,131,2010,"3,066","2,261",805,26.3,Wilcox County,AL


In [7]:
# Turn county and state fips codes into string

sorted_county_df.state_fips_code = sorted_county_df.state_fips_code.astype(str)
sorted_county_df.county_fips_code = sorted_county_df.county_fips_code.astype(str)

In [8]:
# Drop data after decimal point for county_fips_code

sorted_county_df["county_fips_code"]=sorted_county_df["county_fips_code"].str.split(pat=".")
sorted_county_df["county_fips_code"]

16278     [25]
13059     [25]
19497     [25]
9840      [25]
12926    [131]
9709     [131]
22628     [27]
12971     [27]
22716     [25]
29154     [25]
10878     [13]
16190     [27]
32373     [25]
25935     [25]
19409     [27]
25818    [158]
22599    [158]
16145    [131]
29037    [158]
12136    [135]
9752      [27]
13317    [141]
25847     [27]
6623      [25]
19380    [158]
29066     [27]
32256    [158]
10939    [135]
16271     [11]
15210     [69]
         ...  
4853      [97]
27731     [11]
30985     [81]
30955     [21]
32423      [9]
1636      [97]
3798      [81]
8466     [105]
24551     [89]
9560      [35]
21318     [61]
27770     [89]
5240      [87]
21300     [25]
27752     [53]
6343      [35]
27737     [23]
24533     [53]
24537     [61]
21314     [53]
21299     [23]
18121    [105]
24518     [23]
24519     [25]
27756     [61]
27738     [25]
8457      [87]
27778    [105]
24559    [105]
21340    [105]
Name: county_fips_code, Length: 34525, dtype: object

In [9]:
# Drop data after decimal point for county_fips_code

sorted_county_df["state_fips_code"]=sorted_county_df["state_fips_code"].str.split(pat=".")
sorted_county_df["state_fips_code"]

16278     [6]
13059     [6]
19497     [6]
9840      [6]
12926     [1]
9709      [1]
22628     [4]
12971     [4]
22716     [6]
29154     [6]
10878    [26]
16190     [4]
32373     [6]
25935     [6]
19409     [4]
25818     [2]
22599     [2]
16145     [1]
29037     [2]
12136    [47]
9752      [4]
13317    [13]
25847     [4]
6623      [6]
19380     [2]
29066     [4]
32256     [2]
10939    [26]
16271     [6]
15210    [45]
         ... 
4853     [30]
27731    [38]
30985    [38]
30955    [38]
32423     [8]
1636     [30]
3798     [16]
8466     [38]
24551    [38]
9560     [56]
21318    [38]
27770    [38]
5240     [38]
21300    [38]
27752    [38]
6343     [56]
27737    [38]
24533    [38]
24537    [38]
21314    [38]
21299    [38]
18121    [38]
24518    [38]
24519    [38]
27756    [38]
27738    [38]
8457     [38]
27778    [38]
24559    [38]
21340    [38]
Name: state_fips_code, Length: 34525, dtype: object

In [10]:
# Convert county_fips_code to match format of CDC data

sorted_county_df["Updated county_fips_code"] = sorted_county_df["county_fips_code"].apply(lambda x: x[0].zfill(3))
sorted_county_df["Updated county_fips_code"]

16278    025
13059    025
19497    025
9840     025
12926    131
9709     131
22628    027
12971    027
22716    025
29154    025
10878    013
16190    027
32373    025
25935    025
19409    027
25818    158
22599    158
16145    131
29037    158
12136    135
9752     027
13317    141
25847    027
6623     025
19380    158
29066    027
32256    158
10939    135
16271    011
15210    069
        ... 
4853     097
27731    011
30985    081
30955    021
32423    009
1636     097
3798     081
8466     105
24551    089
9560     035
21318    061
27770    089
5240     087
21300    025
27752    053
6343     035
27737    023
24533    053
24537    061
21314    053
21299    023
18121    105
24518    023
24519    025
27756    061
27738    025
8457     087
27778    105
24559    105
21340    105
Name: Updated county_fips_code, Length: 34525, dtype: object

In [11]:
# Convert state_fips_code to match format of CDC data

sorted_county_df["Updated state_fips_code"] = sorted_county_df["state_fips_code"].apply(lambda x: x[0])
sorted_county_df["Updated state_fips_code"]

16278     6
13059     6
19497     6
9840      6
12926     1
9709      1
22628     4
12971     4
22716     6
29154     6
10878    26
16190     4
32373     6
25935     6
19409     4
25818     2
22599     2
16145     1
29037     2
12136    47
9752      4
13317    13
25847     4
6623      6
19380     2
29066     4
32256     2
10939    26
16271     6
15210    45
         ..
4853     30
27731    38
30985    38
30955    38
32423     8
1636     30
3798     16
8466     38
24551    38
9560     56
21318    38
27770    38
5240     38
21300    38
27752    38
6343     56
27737    38
24533    38
24537    38
21314    38
21299    38
18121    38
24518    38
24519    38
27756    38
27738    38
8457     38
27778    38
24559    38
21340    38
Name: Updated state_fips_code, Length: 34525, dtype: object

In [12]:
# Concatenate updated county and state_fips_code into a new combined fips code

new_fips = sorted_county_df["Updated state_fips_code"] + sorted_county_df["Updated county_fips_code"]
new_fips

16278     6025
13059     6025
19497     6025
9840      6025
12926     1131
9709      1131
22628     4027
12971     4027
22716     6025
29154     6025
10878    26013
16190     4027
32373     6025
25935     6025
19409     4027
25818     2158
22599     2158
16145     1131
29037     2158
12136    47135
9752      4027
13317    13141
25847     4027
6623      6025
19380     2158
29066     4027
32256     2158
10939    26135
16271     6011
15210    45069
         ...  
4853     30097
27731    38011
30985    38081
30955    38021
32423     8009
1636     30097
3798     16081
8466     38105
24551    38089
9560     56035
21318    38061
27770    38089
5240     38087
21300    38025
27752    38053
6343     56035
27737    38023
24533    38053
24537    38061
21314    38053
21299    38023
18121    38105
24518    38023
24519    38025
27756    38061
27738    38025
8457     38087
27778    38105
24559    38105
21340    38105
Length: 34525, dtype: object

In [60]:
# Insert new column to contain new_fips data

sorted_county_df['new_fips'] = new_fips
sorted_county_df['new_fips'] = sorted_county_df['new_fips'].astype(int) 
final_county_df = sorted_county_df
final_county_df.head()

,laus_code,state_fips_code,county_fips_code,year,labor_force,employed,unemployed,unemployment_rate,county,state,Updated county_fips_code,Updated state_fips_code,new_fips
16278,CN0602500000000,[6],[25],2011,"79,456","56,524","22,932",28.9,Imperial County,CA,025,6,6025
13059,CN0602500000000,[6],[25],2010,"78,665","55,978","22,687",28.8,Imperial County,CA,025,6,6025
19497,CN0602500000000,[6],[25],2012,"79,265","57,586","21,679",27.4,Imperial County,CA,025,6,6025
9840,CN0602500000000,[6],[25],2009,"75,569","54,889","20,680",27.4,Imperial County,CA,025,6,6025
12926,CN0113100000000,[1],[131],2010,"3,066","2,261",805,26.3,Wilcox County,AL,131,1,1131


In [61]:
cdc_df_2006 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2006.html")[0]
cdc_df_2007 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2007.html")[0]
cdc_df_2008 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2008.html")[0]
cdc_df_2009 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2009.html")[0]
cdc_df_2010 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2010.html")[0]
cdc_df_2011 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2011.html")[0]
cdc_df_2012 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2012.html")[0]
cdc_df_2013 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2013.html")[0]
cdc_df_2014 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2014.html")[0]
cdc_df_2015 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2015.html")[0]
cdc_df_2016 = pd.read_html("https://www.cdc.gov/drugoverdose/maps/rxcounty2016.html")[0]

cdc_concat_df = pd.concat([cdc_df_2006,cdc_df_2007,cdc_df_2008,cdc_df_2009,cdc_df_2010,\
                   cdc_df_2011,cdc_df_2012,cdc_df_2013,cdc_df_2014,cdc_df_2015,cdc_df_2016],axis=1)

cdc_df = cdc_concat_df.groupby(level=0, axis=1).first()
cdc_df.head()

,2006 Prescribing Rate,2007 Prescribing Rate,2008 Prescribing Rate,2009 Prescribing Rate,2010 Prescribing Rate,2011 Prescribing Rate,2012 Prescribing Rate,2013 Prescribing Rate,2014 Prescribing Rate,2015 Prescribing Rate,2016 Prescribing Rate,County,FIPS County Code,State
0,–,–,–,–,–,–,–,–,–,–,–,"Aleutians East, AK",2013,AK
1,–,–,–,–,–,–,–,–,–,–,–,"Aleutians West, AK",2016,AK
2,71.5,75.5,75.6,74.6,75.1,75.4,75.3,68.6,68.0,68.2,66.3,"Anchorage, AK",2020,AK
3,–,–,–,–,–,–,–,–,–,–,–,"Bethel, AK",2050,AK
4,–,–,–,–,–,–,–,–,–,–,–,"Bristol Bay, AK",2060,AK


In [62]:
cdc_df = cdc_concat_df.groupby(level=0, axis=1).first()

cdc_df = cdc_df.rename(columns={'2006 Prescribing Rate': '2006_px', '2007 Prescribing Rate': '2007_px', \
                               '2008 Prescribing Rate': '2008_px','2009 Prescribing Rate': '2009_px',\
                                '2010 Prescribing Rate': '2010_px','2011 Prescribing Rate': '2011_px',\
                               '2012 Prescribing Rate': '2012_px','2013 Prescribing Rate': '2013_px',\
                               '2014 Prescribing Rate': '2014_px','2015 Prescribing Rate': '2015_px',\
                               '2016 Prescribing Rate': '2016_px','FIPS County Code': 'new_fips'})

cdc_df.head()

,2006_px,2007_px,2008_px,2009_px,2010_px,2011_px,2012_px,2013_px,2014_px,2015_px,2016_px,County,new_fips,State
0,–,–,–,–,–,–,–,–,–,–,–,"Aleutians East, AK",2013,AK
1,–,–,–,–,–,–,–,–,–,–,–,"Aleutians West, AK",2016,AK
2,71.5,75.5,75.6,74.6,75.1,75.4,75.3,68.6,68.0,68.2,66.3,"Anchorage, AK",2020,AK
3,–,–,–,–,–,–,–,–,–,–,–,"Bethel, AK",2050,AK
4,–,–,–,–,–,–,–,–,–,–,–,"Bristol Bay, AK",2060,AK


In [63]:
# CDC county column included both county and state abbreviation; eliminate state abbreviation from this column

final_cdc_df = cdc_df

final_cdc_df [['County','junk']]= final_cdc_df.County.str.split(",",expand=True,)

# final_df = final_df[final_df['2006_px_rate'] != '–']
# final_df = final_df[final_df['2016_px_rate'] != '–']

final_cdc_df = final_cdc_df.drop('junk', axis=1)

# Convert new_fips column data to integer 

final_cdc_df['new_fips'] = final_cdc_df['new_fips'].astype(int)

final_cdc_df['new_fips'].dtype

dtype('int32')

In [64]:
# Merge BLS and CDC dataframe into single dataframe on new_fips

merged_final = pd.merge(final_county_df,final_cdc_df , on=['new_fips'], how='inner')
merged_final.head()

,laus_code,state_fips_code,county_fips_code,year,labor_force,employed,unemployed,unemployment_rate,county,state,...,2009_px,2010_px,2011_px,2012_px,2013_px,2014_px,2015_px,2016_px,County,State
0,CN0602500000000,[6],[25],2011,"79,456","56,524","22,932",28.9,Imperial County,CA,...,50.7,55.9,55.0,56.2,59.1,59.5,53.7,52.4,Imperial,CA
1,CN0602500000000,[6],[25],2010,"78,665","55,978","22,687",28.8,Imperial County,CA,...,50.7,55.9,55.0,56.2,59.1,59.5,53.7,52.4,Imperial,CA
2,CN0602500000000,[6],[25],2012,"79,265","57,586","21,679",27.4,Imperial County,CA,...,50.7,55.9,55.0,56.2,59.1,59.5,53.7,52.4,Imperial,CA
3,CN0602500000000,[6],[25],2009,"75,569","54,889","20,680",27.4,Imperial County,CA,...,50.7,55.9,55.0,56.2,59.1,59.5,53.7,52.4,Imperial,CA
4,CN0602500000000,[6],[25],2013,"78,271","58,670","19,601",25.0,Imperial County,CA,...,50.7,55.9,55.0,56.2,59.1,59.5,53.7,52.4,Imperial,CA


In [4]:
# Clean up columns in final dataframe

final_df = merged_final.drop(columns=['laus_code','state_fips_code', 'county_fips_code','labor_force',
                                      'employed','unemployed','Updated county_fips_code','Updated state_fips_code'])

# final_df = final_df.rename(columns={'year_x':'2006', 'labor_force_x':'labor_force_2006',
#                                    'employed_x': 'employed_2006', 'unemployed_x':'unemployed:2006',
#                                    'unemployment_rate_x':'unemployment_rate_2006',
    
#                                    'year_y':'2016','labor_force_y':'labor_force_2016',
#                                    'employed_y': 'employed_2016', 'unemployed_y':'unemployed:2016',
#                                    'unemployment_rate_y':'unemployment_rate_2016'})

final_df

NameError: name 'merged_final' is not defined

In [2]:
# Load dataframe into Mongo database and collection

conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

# declare database
db = client.final_db

# Declare the collection
collection = db.final_db

records = final_df.to_dict('records')
db.myCollection.insert_many(records)

NameError: name 'pymongo' is not defined